# Quant Course
## Monte Carlo method, variance reduction

## Martingale Pricing
    
The price of a product with some payoff function $Payoff$ can be calculated as the expected value of the discounted payoff under risk neutral measure. We assume deterministic interest rate so the discount factor can be moved out from the expected value:

$$ V(t) = e^{-r(T-t)}\mathbb{E}[Payoff(S_T)|F_t] $$

We will work in the Black-Scholes model where the underlying follows 

$$ dS_t = rS_tdt + \sigma S_tdW_t $$


## Estimating the expected payoff

Law of large numbers: Expected value can be estimated with the sample mean

$$ m = E[X] \approx \frac{1}{N}\sum_iX_i $$ 



$$ V(t) = e^{-r(T-t)}\mathbb{E}[Payoff(S_T)] $$

* Generate n independent samples from $S_T$
* Evaluate the payoff function on the samples and take mean

We implement payoff function for each contract so the second step is easy, the complication arise at the first step.

In the general case we dont know the distribution of $S_T$, we only know it's inital value at some time $t$ and we have an assumption on it's dinamic (our market model). How to obtain samples from $S_T$?

## Euler–Maruyama method

consider the following stochastic diff equation:

$$ dS_t = \mu(t, S_t) dt + \sigma(t,S_t)dW_t $$

One can approximate the solution numerically 

* partition the time interval of $[0,T]$ into $N$ subintervals: $\Delta T = T/N$,   $ \quad t_i = i*\Delta T$
* we know the stock price today $S_0$. We approximate the stock price at the next timepoint:
$$ S_{t_{i+1}} = S_{t_i} + \mu(t_i, S_{t_i}) \Delta T + \sigma(t_i,S_{t_i}) \cdot (W_{t_{i+1}} - W_{t_i}) $$
* $W_t$ is a brownian motion, it's increments are independent normals:
$$ W_{t_{i+1}} - W_{t_i} \sim \mathcal{N}(0,t_{i+1}-t_i) $$
$$ W_{t_{i+1}} - W_{t_i} = \sqrt{\Delta T}\epsilon_i, \quad \epsilon_i\sim \mathcal{N}(0,1)$$

# Milstein method

Milstein method improves the accuracy of the Euler method by adding a correction term. 

$$ 
S_{t_{i+1}} = 
S_{t_i} + \underbrace{\mu(t_i, S_{t_i}) \Delta T + \sigma(t_i,S_{t_i}) \cdot (W_{t_{i+1}} - W_{t_i}) }_{\text{Euler step}} 
+\underbrace{\frac{1}{2}(\frac{\partial}{\partial S}\sigma(t_i,S_{t_i})) \sigma(t_i,S_{t_i}) ((W_{t_{i+1}} - W_{t_i})^2-\Delta T)}_{\text{correction}} $$

It requires the derivative of the diffusion term. the correction term vanishes when $\sigma(t,S_t)$ does not depend on the spot and Milstein method become identical to Euler.

Euler: weak convergence order of 1, strong convergence order of 1/2

Milstein: weak convergence order of 1, strong convergence order of 1



## Solution to the SDE in the BS model

In BS model the risky asset follows GBM:

$$ dS_t = rS_t dt + \sigma S_tdW_t $$

For this relatively simple SDE the exact solution is known:

$$ S_t = S_0 e^{(r-\frac{\sigma^2}{2})t+\sigma W_t} $$

If we only need samples from $S_T$ we can simulate it directly, no need for partitioning the time interval and simulate the stock price for (from the contract's point of view) useless time points!

## Random number generation

We will use pseudo random numbers: The generated numbers will be statistically random, but if we fix a seed, always the same numbers will be generated

How many random numbers will be needed?

* For a single path, we will need as many $\epsilon$ as the number of simulation tenors.

* Therefor the total number of $\epsilon$ needed is <b>NumberOfPaths * NumberOfSimulTenors<b>

## Simulation tenors

<img src="./img/timegrid.png" style="margin-left:auto; margin-right:auto; width: 1500px;" />

<img src="./img/mc_code_structure.png" style="margin-left:auto; margin-right:auto; width: 1500px;" />

## Monte Carlo Error

In monte carlo we estimate the PV of the trade by an empirical mean

$$ PV = \frac{1}{N}\sum_{i=1}^NX_i $$ 

PV itself is a random variable here with some variance, therefore there is some uncertanity in the PV.

$$ VAR[PV] = VAR\left[\frac{1}{N}\sum_{i=1}^NX_i\right] = \frac{1}{N^2}\sum_{i=1}^NVAR[X_i] = \frac{1}{N}VAR[X]$$

The standard deviation of PV decreases with $\sqrt{N}$, and we can approximate the 95% confidence interval around PV

$$ \sigma^{MC} = \sqrt{VAR[PV]} = \frac{\sigma^X}{\sqrt{N}} \qquad CI = [PV-1.96*\sigma^{MC},PV+1.96*\sigma^{MC}]$$




## Excercise:

Implement the follwoing methods:
- evolve simulated_spot
- simulate_spot_path

Price a fwd/european option with analytic and MC method

Change the number of simulation paths and see how MC noise changes

In [3]:
import os 
import sys

sys.path.append("..\..\..")

from src.pricer import *
MarketData.initialize()

In [2]:
pwd()

'C:\\Users\\Solymosi Ernő\\PycharmProjects\\QuantCourseBP\\Lectures\\2024-summer_interns\\05_Monte_Carlo'

In [4]:
# import xy

und = Stock.TEST_COMPANY
ls = LongShort.LONG
strike = 1.2
expiry = 1
strike_level = strike * MarketData.get_spot()[und]

model = FlatVolModel(und)

contract_fwd = ForwardContract(und, ls, strike_level, expiry)
contract_opt = EuropeanContract(und, PutCallFwd.PUT, ls, strike_level, expiry)

pricer_fwd_an = ForwardAnalyticPricer(contract_fwd, model, Params())
pricer_opt_an = EuropeanAnalyticPricer(contract_opt, model, Params())

mc_params = MCParams(num_of_path=10000,
                     tenor_frequency=0,
                     antithetic=False,
                     standardize=False,
                     control_variate=False,
                     seed=1,
                     evolve_spot_method=MCNumMethod.EXACT)

pricer_fwd_mc = GenericMCPricer(contract_fwd, model, mc_params)
pricer_opt_mc = GenericMCPricer(contract_opt, model, mc_params)

fv_fwd_an = pricer_fwd_an.calc_fair_value()
fv_fwd_mc = pricer_fwd_mc.calc_fair_value_with_ci()

fv_opt_an = pricer_opt_an.calc_fair_value()
fv_opt_mc = pricer_opt_mc.calc_fair_value_with_ci()

pricer_opt_mc.params.evolve_spot_method = MCNumMethod.EULER
pricer_opt_mc.params.tenor_frequency = 4
fv_opt_mc_e = pricer_opt_mc.calc_fair_value_with_ci()

print("FORWARD AN: " + str(fv_fwd_an))
print("FORWARD MC: " + str(fv_fwd_mc))
print("OPTION AN: " + str(fv_opt_an))
print("OPTION MC: " + str(fv_opt_mc))
print("OPTION MC Euler: " + str(fv_opt_mc_e))

FORWARD AN: -14.147530940085687
FORWARD MC: (-13.60287113415415, (-14.184075580227448, -13.021666688080849))
OPTION AN: 20.594305984211573
OPTION MC: (20.253539136216038, (19.88194754958432, 20.625130722847754))
OPTION MC Euler: (20.17854840821545, (19.80342131539317, 20.55367550103773))


In [5]:
paths = [100, 500, 1000, 5000, 10000, 50000]
for p in paths:
    pricer_opt_mc.params.num_of_paths = p
    print(pricer_opt_mc.calc_fair_value_with_ci())

(18.926448306659, (15.254771279627176, 22.598125333690824))
(18.788700289346277, (17.109178463546453, 20.4682221151461))
(19.361744758496375, (18.169716311663276, 20.553773205329477))
(20.186520940121085, (19.652449353608805, 20.720592526633364))
(20.17854840821545, (19.80342131539317, 20.55367550103773))
(20.298908040225825, (20.129015474967176, 20.46880060548448))


# Variance reduction methods

* Standardizing the normal randoms

* Antithetic random numbers 
    
* Control Variate

## Variance Reduction

### Standardizing:

We generate $n$ independent std normals

$$\epsilon = \{\epsilon_1, ..., \epsilon_n\} \qquad \epsilon_i \sim \mathcal{N}(0,1) \, iid$$

The sample itself wont have exactly 0 mean and 1 std dev, so we can adjust it with the sample's mean and std dev:

$$\epsilon^* = \frac{\epsilon - \mu}{\sigma}$$  

### Antithetic randoms:

Once epsilon is generated, flip all the random's sign and reuse them:

$$ \epsilon^* =  \{\epsilon_1, ..., \epsilon_n, -\epsilon_1, ..., -\epsilon_n\} $$


## Variance Reduction, Control Variate

The estimator given control variate Y and parameter b is
$$ X^{CV} = X + b(Y - E[Y]) $$


The new estimator is still unbiased:

$$ E[X^{CV}] = E[X] + b(E[Y] - E[E[Y]]) = E[X] $$

Let's see the variance of the new estimator:

$$ Var[X^{CV}] = Var[X] + Var[b*(Y-E[Y])] + 2Cov[X,b(Y - E[Y] )] = Var[X] + b^2Var[Y] + 2bCov[X,Y] $$

Once we decided what control variate Y we will use, we can choose b such that the modified estimator's variance is minimised:

with $ b^*=-\frac{Cov[X,Y]}{Var[Y]} $, the variance of the new estimator will decrease

$$ Var[X^{CV}] = Var[X] - \frac{(Cov[X,Y])^2}{Var[Y]} $$


## Variance Reduction, Control Variate

$$ X^{CV} = X + b(Y - E[Y])\qquad b^*=-\frac{Cov[X,Y]}{Var[Y]} $$

For example for options one can choose forwards as control variate:

$$ X = (S_T - K)^+ $$
$$ Y = S_T - K $$

1, How to determine $b^*$?

* We will simulate n realization of $S_T$
* Evaluate the payoffs of $X$ and $Y$ on each $S^{(i)}_T$
* Use these samples $[X^{(1)}, ...]$, $[Y^{(1)}, ...]$  to estimate $Cov[X,Y]$ and $Var[Y]$

2, How to determine $E[Y]$?

We can only use such contracts as control variates, which have analytic pricing formula. In that case, we can create an analytic pricer for the contract and invoke it's calc_fair_value method.


## Excercise

Implement the following methods in generic MC pricer:
- calc_fair_value_with_ci


- apply_control_var_adj
- get_controlvar_helper_pricer (support EuropeanOption contract, use forward as control variate)

Test the various noise reduction methods


In [6]:
numofpath = 10000

mc_params_0 = MCParams(num_of_path=numofpath,
                       tenor_frequency=0,
                       antithetic=False,
                       standardize=False,
                       control_variate=False,
                       seed=1,
                       evolve_spot_method = MCNumMethod.EXACT)

mc_params_1 = MCParams(num_of_path=numofpath,
                       tenor_frequency=0,
                       antithetic=True,
                       standardize=False,
                       control_variate=False,
                       seed=1,
                       evolve_spot_method = MCNumMethod.EXACT)

mc_params_2 = MCParams(num_of_path=numofpath,
                       tenor_frequency=0,
                       antithetic=False,
                       standardize=True,
                       control_variate=False,
                       seed=1,
                       evolve_spot_method = MCNumMethod.EXACT)

mc_params_3 = MCParams(num_of_path=numofpath,
                       tenor_frequency=0,
                       antithetic=False,
                       standardize=False,
                       control_variate=True,
                       seed=1,
                       evolve_spot_method = MCNumMethod.EXACT)

for param in [mc_params_0, mc_params_1, mc_params_2, mc_params_3]:
    pricer_opt_mc = GenericMCPricer(contract_opt, model, param)
    price = pricer_opt_mc.calc_fair_value_with_ci()
    pv = price[0]
    stdDev = (price[1][1]-pv)/1.96
    print("PV: " + str(pv) + "     MonteCarlo StdDev: " + str(stdDev))

PV: 20.253539136216038     MonteCarlo StdDev: 0.18958754419985516
PV: 20.596863881422045     MonteCarlo StdDev: 0.1895946559846217
PV: 20.424781171856452     MonteCarlo StdDev: 0.1900058319266762
PV: 20.559619968109786     MonteCarlo StdDev: 0.09041008744633607


### Excercise - Implementing Asian Option (Average Price Option):

Consider a set of avereging time points: $t_1<t_2<...<t_N=T$.

The payoff of an asian option at time $T$ is $max\{\frac{1}{N}\sum_{i=1}^NS(t_i), K\}$

Implement the contract with uniformly distributed observation points. $\{i * T/num\_obs : i = 1,...,num\_obs\}$, where num_ubs is the number of observations

## Assignments

In both tasks:
- extend the code in the repositories src folder for the implementation part a)
- write the code in jupyter for part b)

<b> 8/1 Implement and test Milstein scheme <b>

a, Implement the Milstein method in the MCMethod class's evolve_simulated_spot method, test it by pricing an option with it and compare the PV with analytic (7p)

b, Compare the termianl distribution of a stock using Euler and Milstein method against the theoretical distribution (lognormal). Demonstrate the convergence of both methods by increasing the number of timesteps (8p)

    
<b>8/2 Improve delta calculation for FlatVol model <b>

In BS model the spot at time t is $ S_t = S_0 e^{(r-\frac{\sigma^2}{2})t+\sigma W_t} $. Knowing this, when we bump the spot we dont have to resimulate the spot, instead we can obtain it by scaling the inital simulation: $ S^{bumped}_t = S_0*(1+\delta) e^{(r-\frac{\sigma^2}{2})t+\sigma W_t} $.

a, implement this delta calculation method for MonteCarlo pricer (10p)

b, compare the value and the calculation time of the improved delta with the default bump and revaluation delta. (5p)
